In [ ]:
import pandas as pd
import numpy as np
from datetime import date
#df=pd.read_table("rna_single_cell_type_tissue.tsv") 

url="https://www.proteinatlas.org/download/rna_single_cell_type_tissue.tsv.zip"
log=date.today()
log=str(log)

df=pd.read_table(url, compression='zip')
df.columns = df.columns.str.replace(' ', '')

#provide list of genes
genes=["TSPAN2","NTSR1","PLAUR"]

def sc_celltype_tissue(Data, Gene, cutoff):
    hpa=Data[((Data.Genename == Gene) & (Data.nTPM >= cutoff ))].groupby(['Tissue','Celltype'])['nTPM'].median().reset_index()
    hpa['log2_nTPM']=np.log2(1+hpa.nTPM)
    hpa_sorted=hpa.sort_values(by='log2_nTPM',ascending=False).groupby('Tissue').nth((0,1,2)).reset_index()
    return(hpa_sorted)

def ploting(hpa_dat, gene):
    import plotly.express as px
    import matplotlib.pyplot as plt
    fig = px.scatter(hpa_plot, x="Tissue", y="log2_nTPM", color="log2_nTPM", size='log2_nTPM', 
    hover_data=['log2_nTPM'], text="Celltype",size_max=18)
    fig.update_traces(textposition='top center')
    fig.update_layout(height=800,width=2000,title_text=gene+" Expression: Top 3 Cell Types per Tissue                     "+"(Date: "+log+")",
    
    
    plot_bgcolor="#F8F8FF",
    font=dict(family="sans-serif",size=12,color='#000000'),hoverlabel=dict(font=dict(family='sans-serif', size=16)))

    fig.update_xaxes(title_text='Tissue')
    fig.update_yaxes(title_text='Log2(1+ median nTPM)')

    fig.write_html(gene+"_"+"Top 3 Celll Types per Tissue.html")
    fig.show()


#HPA sc tissue data, Gene name, nTPM threshold, number of top cell types to select
# provide dataframe which has Tissues, Celltype and log_nTPM
for g in genes:
    hpa_plot=sc_celltype_tissue(df, g, 0.25)
    ploting(hpa_plot, g)